In [151]:
import duckdb
import pandas as pd
from fredapi import Fred
import certifi
import os
from datetime import datetime, timedelta, date
from config import FRED_API_KEY

os.environ['SSL_CERT_FILE'] = certifi.where() #API Key security requirements
fred = Fred(api_key=FRED_API_KEY)
today_date = date.today().strftime('%Y-%m-%d') #Fetch Date
# List of FRED series IDs
series_ids = ['GDP', 
              'GDPC1', 
              'CORESTICKM159SFRBATL', 
              'WM2NS', 
              'UNRATE', 
              'DFF', 
              'DGS10', 
              'PCE', 
              'INDPRO', 
              'MRTSSM44000USS',
              'PPIACO',
              'HOUST']

In [152]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred') as con: 
    display(con.sql(sql_query).df())

,name


#### Data Fetched from the Economic Research Federal Reserve Bank of St. Louis

In [155]:
def fetch_series_data(series_id, start_date, end_date):
    """
    Fetch series data for a given series ID between start_date and end_date.
    
    Parameters:
    - series_id (str): The FRED series ID.
    - start_date (str): The start date for data retrieval.
    - end_date (str): The end date for data retrieval.
    
    Returns:
    - pd.Series: The time series data.
    """
    try:
        series_data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
        return series_data
    except Exception as e:
        print(f"Error fetching series data for {series_id}: {e}")
        return pd.Series([])

def fetch_metadata(series_id):
    """
    Fetch metadata for a given series ID.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - dict: The metadata for the series.
    """
    try:
        metadata = fred.get_series_info(series_id)
        return metadata
    except Exception as e:
        print(f"Error fetching metadata for {series_id}: {e}")
        return {}

def construct_dataframe(series_data, metadata, series_id, fetch_date):
    """
    Construct a DataFrame from series data and metadata.
    
    Parameters:
    - series_data (pd.Series): The time series data.
    - metadata (dict): The metadata for the series.
    - series_id (str): The FRED series ID.
    - fetch_date (str): The date when the data was fetched.
    
    Returns:
    - pd.DataFrame: The constructed DataFrame with data and metadata.
    """
    try:
        df = pd.DataFrame(series_data, columns=['Index']).reset_index().rename(columns={'index': 'Date'})
        df['Name'] = metadata.get('title', '')
        df['SeriesID'] = series_id
        df['Source'] = f'https://fred.stlouisfed.org/series/{series_id}'
        df['Units'] = metadata.get('units', '')
        df['SeasonalAdjustment'] = metadata.get('seasonal_adjustment', '')
        df['Frequency'] = metadata.get('frequency', '')
        df['FREDLastUpdatedDate'] = metadata.get('last_updated', '')
        df['FetchDate'] = fetch_date
        return df
    except Exception as e:
        print(f"Error constructing dataframe for {series_id}: {e}")
        return pd.DataFrame()

def fetch_fred_data(series_ids):
    """
    Fetch data and metadata for a list of FRED series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    
    Returns:
    - list of pd.DataFrame: A list of DataFrames with data and metadata for each series.
    """
    today_date = date.today().strftime('%Y-%m-%d')
    twenty_years_ago = (date.today() - timedelta(days=20*365)).strftime('%Y-%m-%d')
    dataframes = []
    
    for series_id in series_ids:
        series_data = fetch_series_data(series_id, twenty_years_ago, today_date)
        if series_data.empty:
            continue
        metadata = fetch_metadata(series_id)
        df = construct_dataframe(series_data, metadata, series_id, today_date)
        dataframes.append(df)
    
    return dataframes


In [156]:
# Fetch data
dataframes = fetch_fred_data(series_ids)

In [157]:
# Display the fetched DataFrames
for df in dataframes:
    display(df)

,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-01-01,11923.447,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
1,2004-04-01,12112.815,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
2,2004-07-01,12305.307,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
3,2004-10-01,12527.214,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
4,2005-01-01,12767.286,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
75,2022-10-01,26408.405,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
76,2023-01-01,26813.601,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
77,2023-04-01,27063.012,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
78,2023-07-01,27610.128,Gross Domestic Product,GDP,https://fred.stlouisfed.org/series/GDP,Billions of Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-01-01,15248.680,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
1,2004-04-01,15366.850,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
2,2004-07-01,15512.619,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
3,2004-10-01,15670.880,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
4,2005-01-01,15844.727,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
75,2022-10-01,21989.981,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
76,2023-01-01,22112.329,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
77,2023-04-01,22225.350,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13
78,2023-07-01,22490.692,Real Gross Domestic Product,GDPC1,https://fred.stlouisfed.org/series/GDPC1,Billions of Chained 2017 Dollars,Seasonally Adjusted Annual Rate,Quarterly,2024-02-28 07:57:02-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,2.126567,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
1,2004-04-01,2.247883,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
2,2004-05-01,2.228612,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
3,2004-06-01,2.370793,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
4,2004-07-01,2.254997,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
235,2023-10-01,4.882928,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
236,2023-11-01,4.688293,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
237,2023-12-01,4.554396,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13
238,2024-01-01,4.603922,Sticky Price Consumer Price Index less Food an...,CORESTICKM159SFRBATL,https://fred.stlouisfed.org/series/CORESTICKM1...,Percent Change from Year Ago,Seasonally Adjusted,Monthly,2024-03-12 12:02:02-05,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-22,6169.5,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
1,2004-03-29,6167.0,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
2,2004-04-05,6247.4,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
3,2004-04-12,6265.5,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
4,2004-04-19,6255.3,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
1033,2024-01-08,20949.1,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
1034,2024-01-15,20934.1,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
1035,2024-01-22,20803.7,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13
1036,2024-01-29,20751.1,M2,WM2NS,https://fred.stlouisfed.org/series/WM2NS,Billions of Dollars,Not Seasonally Adjusted,"Weekly, Ending Monday",2024-02-27 12:02:02-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,5.8,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
1,2004-04-01,5.6,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
2,2004-05-01,5.6,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
3,2004-06-01,5.6,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
4,2004-07-01,5.5,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
235,2023-10-01,3.8,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
236,2023-11-01,3.7,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
237,2023-12-01,3.7,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13
238,2024-01-01,3.7,Unemployment Rate,UNRATE,https://fred.stlouisfed.org/series/UNRATE,Percent,Seasonally Adjusted,Monthly,2024-03-08 08:03:02-06,2024-03-13


...

,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,8098.8,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
1,2004-04-01,8107.2,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
2,2004-05-01,8176.5,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
3,2004-06-01,8157.6,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
4,2004-07-01,8236.9,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
234,2023-09-01,18791.5,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
235,2023-10-01,18810.6,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
236,2023-11-01,18883.6,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13
237,2023-12-01,19010.3,Personal Consumption Expenditures,PCE,https://fred.stlouisfed.org/series/PCE,Billions of Dollars,Seasonally Adjusted Annual Rate,Monthly,2024-02-29 07:44:01-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,92.5325,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
1,2004-04-01,92.9096,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
2,2004-05-01,93.5838,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
3,2004-06-01,92.8621,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
4,2004-07-01,93.5511,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
234,2023-09-01,103.2096,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
235,2023-10-01,102.3722,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
236,2023-11-01,102.6710,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13
237,2023-12-01,102.6715,Industrial Production: Total Index,INDPRO,https://fred.stlouisfed.org/series/INDPRO,Index 2017=100,Seasonally Adjusted,Monthly,2024-02-15 08:33:03-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,286209.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
1,2004-04-01,282952.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
2,2004-05-01,288252.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
3,2004-06-01,284133.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
4,2004-07-01,287358.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
233,2023-08-01,608386.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
234,2023-09-01,612633.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
235,2023-10-01,610561.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13
236,2023-11-01,609099.0,Retail Sales: Retail Trade,MRTSSM44000USS,https://fred.stlouisfed.org/series/MRTSSM44000USS,Millions of Dollars,Seasonally Adjusted,Monthly,2024-02-15 09:07:01-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,143.100,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
1,2004-04-01,144.800,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
2,2004-05-01,146.800,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
3,2004-06-01,147.200,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
4,2004-07-01,147.400,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
234,2023-09-01,258.934,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
235,2023-10-01,255.121,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
236,2023-11-01,253.063,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13
237,2023-12-01,249.767,Producer Price Index by Commodity: All Commodi...,PPIACO,https://fred.stlouisfed.org/series/PPIACO,Index 1982=100,Not Seasonally Adjusted,Monthly,2024-02-16 07:53:01-06,2024-03-13


,Date,Index,Name,SeriesID,Source,Units,SeasonalAdjustment,Frequency,FREDLastUpdatedDate,FetchDate
0,2004-03-01,1998.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
1,2004-04-01,2003.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
2,2004-05-01,1981.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
3,2004-06-01,1828.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
4,2004-07-01,2002.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
...,...,...,...,...,...,...,...,...,...,...
234,2023-09-01,1356.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
235,2023-10-01,1376.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
236,2023-11-01,1512.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
237,2023-12-01,1562.0,New Privately-Owned Housing Units Started: Tot...,HOUST,https://fred.stlouisfed.org/series/HOUST,Thousands of Units,Seasonally Adjusted Annual Rate,Monthly,2024-02-16 07:32:02-06,2024-03-13
